In [1]:
!pip install -U torch datasets transformers

distutils: /opt/anaconda3/include/python3.8/UNKNOWN
sysconfig: /opt/anaconda3/include/python3.8
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 143.1 MB 2.6 MB/s eta 0:00:01    |███▍                            | 15.1 MB 1.9 MB/s eta 0:01:07     |███████▍                        | 33.2 MB 2.9 MB/s eta 0:00:38     |███████████████▉                | 70.6 MB 4.0 MB/s eta 0:00:19     |████████████████████            | 89.1 MB 3.0 MB/s eta 0:00:18     |█████████████████████████▊      | 115.1 MB 2.8 MB/s eta 0:00:10     |███████████████████████████▎    | 122.2 MB 1.5 MB/s eta 0:00:15     |███████████████████████████████ | 138.9 MB 2.9 MB/s eta 0:00:02
     |████████████████████████████████| 474 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 7.1 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████

In [8]:
import json

with open('skin_cancer_dataset.json') as user_file:
    ds = user_file.read()
    
parsed_json = json.loads(ds)
print(parsed_json)

[{'id': 0, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-2-open-sore-340x240.png', 'cancer_img_caption': 'An open sore that does not heal', 'caption_speech': 'id_0.mp3'}, {'id': 1, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/BCC-slider-1.jpg', 'cancer_img_caption': 'An open sore that bleeds, oozes, or crusts and remains open for a few weeks, only to heal up and then bleed again. ', 'caption_speech': 'id_1.mp3'}, {'id': 2, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/BCC-slider-2.jpg', 'cancer_img_caption': 'A reddish patch or irritated area', 'caption_speech': 'id_2.mp3'}, {'id': 3, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-1-shiny-bump-340x240.png', 'cancer_img_caption': 'A shiny bump or nodule', 'cap

In [9]:
type(parsed_json)

list

In [17]:
import requests
image_url = "https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-2-open-sore-340x240.png"
img_data = requests.get(image_url).content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)

In [20]:
import requests

headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Accept-Language": "en-US,en;q=0.9"
           }

for i,ele in enumerate(parsed_json):
    print(f"{i}  -> {ele['cancer_img']}")
    image_url = ele['cancer_img']
    img_data = requests.get(url=image_url, headers=headers).content
    with open(f'{i}.jpg', 'wb') as handler:
        handler.write(img_data)


0  -> https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-2-open-sore-340x240.png
1  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-1.jpg
2  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-2.jpg
3  -> https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-1-shiny-bump-340x240.png
4  -> https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-4-scar-340x240.png
5  -> https://www.skincancer.org/wp-content/uploads/BCC_pigmented_Fitz_type_4_jpg_Bwr4Nu1C-340x240.jpg
6  -> https://www.skincancer.org/wp-content/uploads/bcclatinoman_sm.jpg
7  -> https://www.skincancer.org/wp-content/uploads/BCC_PinkGrowth-1-340x240.jpg
8  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-4.jpg
9  -> https://www.skincancer.org/wp-content/uploads/Pigmented-BCC-behind-the-ear-340x240.jpeg
10  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-5.jpg
11  -> https://www.skincancer.org/wp-content/uploads/Pigmented-BCC-340x240.jpeg
12  ->

In [ ]:
from datasets import load_dataset

imagenette = load_dataset(
    'frgfm/imagenette',
    'full_size',
    split='train',
    ignore_verifications=False  # set to True if seeing splits Error
)
imagenette

In [ ]:

from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
import torch

# if you have CUDA or MPS, set it to the active device like this
device = "cuda" if torch.cuda.is_available() else \
         ("mps" if torch.backends.mps.is_available() else "cpu")
model_id = "openai/clip-vit-base-patch32"

# we initialize a tokenizer, image processor, and the model itself
tokenizer = CLIPTokenizerFast.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id).to(device)


# create text embedding
prompt = "a dog in the snow"

# create transformer-readable tokens
inputs = tokenizer(prompt, return_tensors="pt")
print(inputs)

# Then feed these tokens into the model using the get_text_features method.
# use CLIP to encode tokens into a meaningful embedding
text_emb = model.get_text_features(**inputs)
print(text_emb.shape)


# create image embeddings
image = processor(
    text=None,
    images=imagenette[0]['image'],
    return_tensors='pt'
)['pixel_values'].to(device)
print(image.shape)
import matplotlib.pyplot as plt
# we can still visualize the processed image
plt.imshow(image.squeeze(0).T)

# we process these inputs with CLIP, this time using the get_image_features method.
img_emb = model.get_image_features(image)
print(img_emb.shape)


# And with that, we built vector embeddings for text and image with CLIP. With these embeddings,
# we can compare their similarity using metrics like Euclidean distance, cosine similarity, or dot product similarity.
